In [ ]:
pip install pyserial

In [3]:
import serial
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
#Load the trained model
model = joblib.load('random_forest_model.pkl')

def parse_serial_data(data):
    #Split the data and convert to float values
    values =list(map(float, data.strip().split(',')))
    return np.array(values).reshape(1, -1)

#Open the serial port
ser= serial.Serial('COM4', 9600, timeout=1)
print("Serial port opened successfully.")

try:
    while True:
        if ser.in_waiting > 0:
            #Read the data from the serial port
            data = ser.readline().decode('utf-8').rstrip()
            if data:
                print(f"Received data: {data}")
            # Parse the sensor data
            sensor_data = parse_serial_data(data)
            #Make a prediction using the ML model
            prediction = model.predict(sensor_data)
            #Send the prediction back to the Arduino
            ser.write(f"{int(prediction[0])}\n".encode('utf-8'))
            print(f"Prediction: {prediction}")
            #Small delay to allow Arduino to process the data
            time.sleep(1)

except KeyboardInterrupt:
    print("Closing the serial port...")
finally:
    ser.close()
    print("Serial port closed.")
    

        